In [1]:
from selenium import webdriver
import time
import keyboard
import getpass
import pandas as pd
import requests

In [52]:
#for hashtags
def Instagram(username,pwd,keyword,num_iter): # keyword is the hashtag to extract and num_iter how much data you want to extract
    ###################################### LOGIN #####################################
    if num_iter==0:
        return print(f'User wants nothing to scrape')
    
    driver = webdriver.Chrome('chromedriver')
    driver.maximize_window()
    driver.get('https://www.instagram.com/')
    time.sleep(3)

    driver.find_element_by_xpath("//div[@class='EPjEi']/form/div[2]/div").click()

    keyboard.write(username)
    keyboard.press_and_release('tab') # go to pwd
    time.sleep(2)
    keyboard.write(pwd)
    keyboard.press_and_release('tab')# go to show button(not necessary)
    keyboard.press_and_release('tab')#go to login button
    keyboard.press_and_release('enter')#press enter to login
    time.sleep(5)
    ###################################### SEARCH FOR KEYWORD ####################################
    driver.find_element_by_xpath("//div[@class='LWmhU _0aCwM']/div").click()
    keyboard.write(keyword)
    time.sleep(2)
    keyboard.press_and_release('enter')#enter to go to 1st keyword of list
    keyboard.press_and_release('enter')# select the first keyword from dropdown
    time.sleep(4)
    ###################################### pulling out data from each post(Most recent section) ##########################
    pic_idx = 1 
    row_idx = 1
    pic_count=1
    l1=[]
    post_comment_username_list=[]
    post_comments_list=[]
    each_post_username=[]
    each_post_comment=[]
    img_src_list=[]
    img_alt_list=[]
    post_username_list=[]
    post_caption_list=[]
    count_comments_list=[]
    post_likes_list=[]
    enter=True
    while True:
        time.sleep(1)
        if pic_count == num_iter:
            Insta_dict={'Image Source':img_src_list,'Image Alternate':img_alt_list,'posted by':post_username_list,'Caption':post_caption_list,
                        'No. of (Comments+replies)':count_comments_list,'No. of likes':post_likes_list,'Users who commented':post_comment_username_list,
                        'Comments on post':post_comments_list}
            Insta_df = pd.DataFrame(Insta_dict)
            #hashtag logic
            i=0
            sent = ""
            hashtags=[]
            while i!=len(Insta_df['Caption']):
                hashtag = Insta_df['Caption'].iloc[i]
                for each in hashtag.split():
                    if each.startswith('#'):
                        sent+=each
                if len(sent)>0:
                    hashtags.append(sent)
                else:
                    hashtags.append('None')
                sent=""    
                i+=1
            Insta_df['post hashtags'] = hashtags
            
            #dates logic
            j=0
            dates = []
            while j!=len(Insta_df['Image Alternate']):
                data = Insta_df['Image Alternate'].iloc[j]
                start = data.find('on')+3 # plus 3 to exclue 'on '
                end = data.find('.')
                dates.append(data[start:end])
                j+=1
            Insta_df['Date'] = dates    
            return driver,Insta_df


        else:

            if enter: # get only first pic of entered #hashtag
                                                                
                enter=False                   
                driver.find_element_by_xpath(f"/html[1]/body[1]/div[1]/section[1]/main[1]/article[1]/div[2]/div[1]/div[1]/div[1]").click() #iterate with i at last div to contol columns and second last div with j to control rows 

            else:#after getting first pic just press tab to go onto next pic
                keyboard.press_and_release('tab')
                keyboard.press_and_release('enter')
            time.sleep(3)

            #for img scrapping
            try:
                img = driver.find_element_by_xpath("//div[@class='_97aPb ']/div/div/div/img")
            except:#if the post is a video
                pic_idx+=1
                keyboard.press_and_release('esc')
                continue
                
            img_src = img.get_attribute("src")
            img_alt = img.get_attribute("alt")
            print(img_src)
            print(img_alt)


            time.sleep(1)    
            post_username = driver.find_elements_by_xpath("/html[1]/body[1]/div[4]/div[2]/div[1]/article[1]/header[1]/div[2]/div[1]/div[1]/a[1]")[0].get_attribute('text')    
            print(post_username)
            post_caption = driver.find_elements_by_xpath("/html[1]/body[1]/div[4]/div[2]/div[1]/article[1]/div[2]/div[1]/ul[1]/div[1]/li[1]/div[1]/div[1]/div[2]/span[1]")[0].text
            print(post_caption)
            
            #comments are in ul tags so I counted the no. of ul tags that gives the total number of comments
            ul_tag = driver.find_elements_by_xpath("/html[1]/body[1]/div[4]/div[2]/div[1]/article[1]/div[2]/div[1]/ul[1]")[0]
            count_comments = len(ul_tag.find_elements_by_tag_name('ul'))
            print(f'count_comments  {count_comments}')
            l1.append(count_comments)
            
            try:
                post_likes = driver.find_elements_by_xpath("/html[1]/body[1]/div[4]/div[2]/div[1]/article[1]/div[2]/section[2]/div[1]/div[1]/button[1]/span[1]")[0].text

            except:
                #if no likes on post
                post_likes=0
            print(f'post_likes  {post_likes}')
            
            img_src_list.append(img_src)
            img_alt_list.append(img_alt)
            post_username_list.append(post_username)
            post_caption_list.append(post_caption)
            count_comments_list.append(count_comments)
            post_likes_list.append(post_likes)
            
            if count_comments > 0:# if there are some comments then append them to list
                for i in range(1,count_comments):
                    try:
                        print(i)
                        #put i in ul tag 2nd from starting
                        post_comment_username = driver.find_elements_by_xpath(f"/html[1]/body[1]/div[4]/div[2]/div[1]/article[1]/div[2]/div[1]/ul[1]/ul[{i}]/div[1]/li[1]/div[1]/div[1]/div[2]/h3[1]/div[1]/a[1]")[0].text
                        each_post_username.append(post_comment_username)

                        post_comments = driver.find_elements_by_xpath(f"/html[1]/body[1]/div[4]/div[2]/div[1]/article[1]/div[2]/div[1]/ul[1]/ul[{i}]/div[1]/li[1]/div[1]/div[1]/div[2]/span[1]")[0].text
                        each_post_comment.append(post_comments)


                        print(post_comment_username)
                        print(post_comments)
                    except:
                        continue
                post_comment_username_list.append(each_post_username)    
                post_comments_list.append(each_post_comment)
                
            else: #append none if no comments
                post_comment_username_list.append('None')
                post_comments_list.append('-')

                
                


            print("#####################################################################################################")
            pic_count+=1
            pic_idx+=1
            keyboard.press_and_release('esc')# to close the open post
            ###################################################### Save to Database ##########################################

        

In [2]:
username='_fitness_spartans'
pwd = getpass.getpass()
keyword = '#moneyhiest'


········


In [49]:
num_iter = int(input('no. of pics to scrape  '))
driver,Insta_df= Instagram(username,pwd,keyword,num_iter)


no. of pics to scrape  30
https://instagram.fixc2-1.fna.fbcdn.net/v/t51.2885-15/e35/103572276_259209365339149_8573934078115289317_n.jpg?_nc_ht=instagram.fixc2-1.fna.fbcdn.net&_nc_cat=101&_nc_ohc=fRn6dPznGhUAX8dq2EK&oh=4866e240902b36558264245a0b0131b5&oe=5F130348
Photo by •Stay Home ~ MUM• on June 15, 2020. Image may contain: 4 people, text
maamulgaundadhu_memes
Check cheskondi kavalante 😌
.
.
@dharidhram
#MUM_posts
#raghuvaranbtech
#dhanush
#moneyhiest
#vishwaksen
#maheshbabu
#manchumanoj
#sharwanand
#pubg
#anupamaparameswaran
#kajal
#mahesbabu
#raviteja
#pavankalyan
#chiranjeevi
#blm
#ranadaggubati
#prabhas
#rampothineni
#sarkaruvaripaata
#anushka
#mindblock
#alluarjun
#adhurs
#brahmanandam
#ali
#cakevideos
#saipallavi
count_comments  13
post_likes  32
1
pavidisney
#Gay #Lesbian #LoveWins #Pride #PrideMonth Ipovu varum choodu 😂😂
2
supriya_2936
#pride2020
3
mallik_arjun___008
#equality
4
dharidhram
#lgbtq
5
anupama_lovers_
#lovewins
6
manojkissy
#equality
7
sr_srividyareddy93
#equality

bigpoppa_deja
Why does joey look so good 😂
3
breewinfield._
joey and monica
4
xo.maria_x
Joey, Monica and Rachel all look GOOD tbh😂
5
obada_zaher
Chandler looks like his father😂💔
6
megpraj1
Jen kinda looks like Brad
7
felfeli359
Definitely Joey
8
analy_mirzaei
Joey😂
9
anantvaibhav707
My eyes my eyes😂😂😂
10
hats.2008
Monica 😂
11
kimiaansri
Joey's hot !😂
12
kuldona__nini
Yack🤢🤮
#####################################################################################################
https://instagram.fixc2-1.fna.fbcdn.net/v/t51.2885-15/e35/81990153_1385973064924914_8387226730890003885_n.jpg?_nc_ht=instagram.fixc2-1.fna.fbcdn.net&_nc_cat=106&_nc_ohc=G706YEz34eYAX8ZYsT-&oh=35c2f49dce2cfb5d3de9f10fe686c318&oe=5F116B60
Photo by Ritzie🌼 on June 15, 2020. Image may contain: text
ritziebloom
#moneyhiest #robbery #netflix #songs #music #lockdownmood
count_comments  0
post_likes  0
#####################################################################################################
https://instagram.fi

maamulgaundadhu_memes
🤧🤧 ippudu kuda alane anipisthadhi anukondi
.
.
#MUM_posts
#raghuvaranbtech
#dhanush
#moneyhiest
#nibbanibbimemes
#maheshbabu
#manchumanoj
#sharwanand
#pubg
#anupamaparameswaran
#kajal
#mahesbabu
#raviteja
#pavankalyan
#chiranjeevi
#akhil
#ranadaggubati
#prabhas
#rampothineni
#sarkaruvaripaata
#anushka
#mindblock
#alluarjun
#adhurs
#brahmanandam
#ali
#cakevideos
#saipallavi
#davidwarner
#iamkalpika
count_comments  0
post_likes  68
#####################################################################################################
https://instagram.fixc2-1.fna.fbcdn.net/v/t51.2885-15/e35/104245340_307465016952914_3374859321349757334_n.jpg?_nc_ht=instagram.fixc2-1.fna.fbcdn.net&_nc_cat=102&_nc_ohc=SJCtn_BlU2kAX_bCwGd&oh=a55267697a4915141b8d11964faba9de&oe=5F12E53B
Photo by rabia on June 14, 2020. Image may contain: 15 people, meme and text
rabia_riyazz
#moneyhiest
count_comments  1
post_likes  7
#######################################################################

kino.info_
Նշեք ձեր ամենասիրելի կերպարին😎

#ֆիլմ #կինո #կինոման #lacasadepapel #moneyhiest #netflix #amc #breakingbad #bbc #peakyblinders #foxtv #prisonbreak #history #vikings #սուրհովարներ #թղթետունը #փախուստբանտից
count_comments  0
post_likes  19
#####################################################################################################
https://instagram.fixc2-1.fna.fbcdn.net/v/t51.2885-15/e35/103830150_570772273861489_4688618265595698299_n.jpg?_nc_ht=instagram.fixc2-1.fna.fbcdn.net&_nc_cat=101&_nc_ohc=qy0RS1xjiooAX956u2b&oh=80261179cab384d6c0d77fb000075f31&oe=5F11AB92
Photo by maulik shrimali in India. Image may contain: 2 people
mr.shrimali_143
when people tell you you've changed, it's only because you stopped acting the way they want you to act.🤨💫✨
#mscreation143
count_comments  17
post_likes  42
1
sahil_amin._
Samjme nai aaya lekin sunke achha laga...🙂
2
_mr_j_star_
Layo layo navu😎
3
_mr_j_star_
Chamkyo bhai
4
missbittu07
Indian🇮🇳👳..
5
iswar____damor____04
😍😍
6
iswar___

In [50]:
Insta_df

,Image Source,Image Alternate,posted by,Caption,No. of (Comments+replies),No. of likes,Users who commented,Comments on post,post hashtags,Date
0,https://instagram.fixc2-1.fna.fbcdn.net/v/t51....,"Photo by •Stay Home ~ MUM• on June 15, 2020. I...",maamulgaundadhu_memes,Check cheskondi kavalante 😌\n.\n.\n@dharidhram...,13,32,"[pavidisney, supriya_2936, mallik_arjun___008,...",[#Gay #Lesbian #LoveWins #Pride #PrideMonth Ip...,#MUM_posts#raghuvaranbtech#dhanush#moneyhiest#...,"June 15, 2020"
1,https://instagram.fixc2-1.fna.fbcdn.net/v/t51....,"Photo by Drunk Bunny🧸💫 on June 15, 2020. Image...",whosmeir2.0,#moneyhiest only you will understand,0,0,None,-,#moneyhiest,"June 15, 2020"
2,https://instagram.fixc2-1.fna.fbcdn.net/v/t51....,Photo by Lucifer in FindOut. Image may contain...,unkwn_for_ya,Berlin👺\n#berlin\n#moneyhiest\n#netflix,0,2,None,-,#berlin#moneyhiest#netflix,
3,https://instagram.fixc2-1.fna.fbcdn.net/v/t51....,"Photo by mobi pics❤️ on June 15, 2020.",_mobile_photography360,Double mask 😍\n#photooftheday #photographers_o...,3,47,"[pavidisney, supriya_2936, mallik_arjun___008,...",[#Gay #Lesbian #LoveWins #Pride #PrideMonth Ip...,#photooftheday#photographers_of_india#moneyhie...,"June 15, 2020"
4,https://instagram.fixc2-1.fna.fbcdn.net/v/t51....,"Photo by •Stay Home ~ MUM• on June 15, 2020. I...",maamulgaundadhu_memes,🤤❤️\n.\n.\n.\n.\n.\n.\n.\n#MUM_posts\n#raghuva...,0,47,None,-,#MUM_posts#raghuvaranbtech#dhanush#moneyhiest#...,"June 15, 2020"
5,https://instagram.fixc2-1.fna.fbcdn.net/v/t51....,"Photo by UwU on June 15, 2020. Image may conta...",brainless_mia,I loved Jaime Lorente's role as Denver in Mone...,2,19,"[pavidisney, supriya_2936, mallik_arjun___008,...",[#Gay #Lesbian #LoveWins #Pride #PrideMonth Ip...,#denver#moneyhiest#moneyheistnetflix#denvermon...,"June 15, 2020"
6,https://instagram.fixc2-1.fna.fbcdn.net/v/t51....,"Photo by •Stay Home ~ MUM• on June 15, 2020. I...",maamulgaundadhu_memes,Saradha just for fun😪🚶🏻‍♀️🚶🏻‍♀️🚶🏻‍♀️🚶🏻‍♀️\n.\n...,8,75,"[pavidisney, supriya_2936, mallik_arjun___008,...",[#Gay #Lesbian #LoveWins #Pride #PrideMonth Ip...,#MUM_posts#raghuvaranbtech#dhanush#moneyhiest#...,"June 15, 2020"
7,https://instagram.fixc2-1.fna.fbcdn.net/v/t51....,"Photo by Bright Screen⭐ on June 15, 2020. Imag...",brightscreen_ir,"...\n#دوستاتو_تگ_کن\n🎬دیالوگ ۳\n.\n🔸برلین:\n""ع...",6,152,"[pavidisney, supriya_2936, mallik_arjun___008,...",[#Gay #Lesbian #LoveWins #Pride #PrideMonth Ip...,#دوستاتو_تگ_کن#خانه_کاغذی#سرقت_پول,"June 15, 2020"
8,https://instagram.fixc2-1.fna.fbcdn.net/v/t51....,"Photo by @friends.ios on June 15, 2020. Image ...",friends.ios,Which friend can pull off being another gender...,13,"3,360","[pavidisney, supriya_2936, mallik_arjun___008,...",[#Gay #Lesbian #LoveWins #Pride #PrideMonth Ip...,#friendstv#friendsedit#friendstvseries#friends...,
9,https://instagram.fixc2-1.fna.fbcdn.net/v/t51....,"Photo by Ritzie🌼 on June 15, 2020. Image may c...",ritziebloom,#moneyhiest #robbery #netflix #songs #music #l...,0,0,None,-,#moneyhiest#robbery#netflix#songs#music#lockdo...,"June 15, 2020"


In [76]:
def Insta_urls(username,pwd,url):
    ###################################### LOGIN #####################################    
    driver = webdriver.Chrome('chromedriver')
    driver.maximize_window()
    driver.get('https://www.instagram.com/')
    time.sleep(3)

    driver.find_element_by_xpath("//div[@class='EPjEi']/form/div[2]/div").click()

    keyboard.write(username)
    keyboard.press_and_release('tab') # go to pwd
    time.sleep(2)
    keyboard.write(pwd)
    keyboard.press_and_release('tab')# go to show button(not necessary)
    keyboard.press_and_release('tab')#go to login button
    keyboard.press_and_release('enter')#press enter to login
    time.sleep(5)
    #################################### GO TO URL ###################################
    driver.get(url)
    ################################## scrapping #####################################
    ######## account info #############
    pic_idx = 1 
    row_idx = 1
    pic_count=1
    l1=[]
    post_comment_username_list=[]
    post_comments_list=[]
    each_post_username=[]
    each_post_comment=[]
    img_src_list=[]
    img_alt_list=[]
    post_username_list=[]
    post_caption_list=[]
    count_comments_list=[]
    post_likes_list=[]
    enter = True
    
    start_idx = url.find('.com/')+5
    username = url[start_idx:-1]
    posts = driver.find_elements_by_xpath('/html[1]/body[1]/div[1]/section[1]/main[1]/div[1]/header[1]/section[1]/ul[1]/li[1]/span[1]/span[1]')[0].text
    followers = driver.find_elements_by_xpath('/html[1]/body[1]/div[1]/section[1]/main[1]/div[1]/header[1]/section[1]/ul[1]/li[2]/a[1]/span[1]')[0].text
    following = driver.find_elements_by_xpath('/html[1]/body[1]/div[1]/section[1]/main[1]/div[1]/header[1]/section[1]/ul[1]/li[3]/a[1]/span[1]')[0].text
    name = driver.find_elements_by_xpath('/html[1]/body[1]/div[1]/section[1]/main[1]/div[1]/header[1]/section[1]/div[2]/h1[1]')[0].text
    bio = driver.find_elements_by_xpath('/html[1]/body[1]/div[1]/section[1]/main[1]/div[1]/header[1]/section[1]/div[2]/span[1]')[0].text
    
    print(f'posts: {posts}   followers: {followers}  following:{following} username: {username}  name: {name}  bio: {bio}')
    ########### post info ###########
    
    while True:
        time.sleep(1)
        if pic_count == 20:#int(posts)
            Insta_dict={'Image Source':img_src_list,'Image Alternate':img_alt_list,'posted by':post_username_list,'Caption':post_caption_list,
                        'No. of (Comments+replies)':count_comments_list,'No. of likes':post_likes_list,'Users who commented':post_comment_username_list,
                        'Comments on post':post_comments_list}
            Insta_df= pd.DataFrame(Insta_dict)
            #hashtag logic
            i=0
            sent = ""
            hashtags=[]
            while i!=len(Insta_df['Caption']):
                hashtag = Insta_df['Caption'].iloc[i]
                for each in hashtag.split():
                    if each.startswith('#'):
                        sent+=each
                if len(sent)>0:
                    hashtags.append(sent)
                else:
                    hashtags.append('None')
                sent=""    
                i+=1
            Insta_df['post hashtags'] = hashtags
            
            #dates logic
            j=0
            dates = []
            while j!=len(Insta_df['Image Alternate']):
                data = Insta_df['Image Alternate'].iloc[j]
                start = data.find('on')+3 # plus 3 to exclue 'on '
                end = data.find('.')
                dates.append(data[start:end])
                j+=1
            Insta_df['Date'] = dates  
#             Insta_df_url = Insta_df 
            return driver,Insta_df


        else:

            if enter: # get only first pic of entered #hashtag
                                                                
                enter=False                   
                driver.find_element_by_xpath(f"/html[1]/body[1]/div[1]/section[1]/main[1]/div[1]/div[3]/article[1]/div[1]/div[1]/div[1]/div[1]").click() #iterate with i at last div to contol columns and second last div with j to control rows 

            else:#after getting first pic just press tab to go onto next pic
                keyboard.press_and_release('tab')
                keyboard.press_and_release('enter')
            time.sleep(3)

            #for img scrapping
            try:
                img = driver.find_element_by_xpath("/html[1]/body[1]/div[4]/div[2]/div[1]/article[1]/div[1]/div[1]/div[1]/div[1]/div[1]/img[1]")
            except:#if the post is a video
                pic_idx+=1
                keyboard.press_and_release('esc')
                continue
                
            img_src = img.get_attribute("src")
            img_alt = img.get_attribute("alt")
            print(img_src)
            print(img_alt)


            time.sleep(1)    
            post_username = driver.find_elements_by_xpath("/html[1]/body[1]/div[4]/div[2]/div[1]/article[1]/header[1]/div[2]/div[1]/div[1]/a[1]")[0].get_attribute('text')    
            print(post_username)
            try:
                post_caption = driver.find_elements_by_xpath("/html[1]/body[1]/div[4]/div[2]/div[1]/article[1]/div[2]/div[1]/ul[1]/div[1]/li[1]/div[1]/div[1]/div[2]/span[1]")[0].text
            except:
                post_caption = 'User did not put any caption'
            print(post_caption)
            
            #comments are in ul tags so I counted the no. of ul tags that gives the total number of comments
            ul_tag = driver.find_elements_by_xpath("/html[1]/body[1]/div[4]/div[2]/div[1]/article[1]/div[2]/div[1]/ul[1]")[0]
            count_comments = len(ul_tag.find_elements_by_tag_name('ul'))
            print(f'count_comments  {count_comments}')
            l1.append(count_comments)
            
            try:
                post_likes = driver.find_elements_by_xpath("/html[1]/body[1]/div[4]/div[2]/div[1]/article[1]/div[2]/section[2]/div[1]/div[1]/button[1]/span[1]")[0].text

            except:
                #if no likes on post
                post_likes=0
            print(f'post_likes  {post_likes}')
            
            img_src_list.append(img_src)
            img_alt_list.append(img_alt)
            post_username_list.append(post_username)
            post_caption_list.append(post_caption)
            count_comments_list.append(count_comments)
            post_likes_list.append(post_likes)
            
            if count_comments > 0:# if there are some comments then append them to list
                for i in range(1,count_comments):
                    try:
                        print(i)
                        #put i in ul tag 2nd from starting
                        post_comment_username = driver.find_elements_by_xpath(f"/html[1]/body[1]/div[4]/div[2]/div[1]/article[1]/div[2]/div[1]/ul[1]/ul[{i}]/div[1]/li[1]/div[1]/div[1]/div[2]/h3[1]/div[1]/a[1]")[0].text
                        each_post_username.append(post_comment_username)

                        post_comments = driver.find_elements_by_xpath(f"/html[1]/body[1]/div[4]/div[2]/div[1]/article[1]/div[2]/div[1]/ul[1]/ul[{i}]/div[1]/li[1]/div[1]/div[1]/div[2]/span[1]")[0].text
                        each_post_comment.append(post_comments)


                        print(post_comment_username)
                        print(post_comments)
                    except:
                        continue
                post_comment_username_list.append(each_post_username)    
                post_comments_list.append(each_post_comment)
                
            else: #append none if no comments
                post_comment_username_list.append('None')
                post_comments_list.append('-')

                
                

            print(f'pic_count: {pic_count}')
            print("#####################################################################################################")
            
            pic_count+=1
            pic_idx+=1
            
            keyboard.press_and_release('esc')# to close the open post

In [77]:
url = 'https://www.instagram.com/leomessi/'
driver,Insta_df_url = Insta_urls(username,pwd,url)

posts: 601   followers: 155m  following:230 username: leomessi  name: Leo Messi  bio: Bienvenidos a la cuenta oficial de Instagram de Leo Messi / Welcome to the official Leo Messi Instagram account
https://instagram.fixc2-1.fna.fbcdn.net/v/t51.2885-15/e35/83936572_2800767210053104_78195178703949855_n.jpg?_nc_ht=instagram.fixc2-1.fna.fbcdn.net&_nc_cat=1&_nc_ohc=tiiNL7eoDsEAX8qEt-P&oh=88b0500c689c61fe40591e9381420d89&oe=5F11E054
Photo shared by Leo Messi on June 13, 2020 tagging @3gerardpique, @antogriezmann, @jordialbaoficial, @luissuarez9, and @kingarturo23oficial. Image may contain: 2 people
leomessi
✈️📍Mallorca - @laliga
count_comments  24
post_likes  6,387,075
1
carles5puyol
💪💪💪💪
2
themessistore
🐐🐐🐐
3
laliga
🐐😷🔥⚽️🔜
4
thetrillionairelife
Best Soccer Player ⚽️❤️
5
thetrillionairelife
Greatest of All Time 🐐
6
thetrillionairelife
G.O.A.T 🐐
7
thetrillionairelife
⚽️⚽️⚽️❤️❤️❤️
8
laliga
🔥🔥
9
rahamasadau
💃🏻
10
elizabethloaiza
👌🏻😷👍🏻🙏🏻
11
jordiielias
Hoy vuelve Su Majestad😍😍
12
bin_baz
اسطورة


leomessi
@championsleague ⚽️🏟
count_comments  12
post_likes  4,193,344
1
mergenbabayew8
👏
2
sodiqjonrafiqjonov3
GOOD 👍👍👍👍
3
frxncisco14
Comenten mi ultima publicacion como si fueramos panas :c
4
beleckytte
Love you Messi! ❤️🔥
5
kostiatoroptsev
🏆🥇
6
hoba.2003
❤️❤️😍
7
rayssanatashamarques
Sempre toso por você
8
matheusamorim2012
Ídolo ⚽️⚽️⚽️🔥🏆
9
abdi_kam
🔥
10
cristian.1_5
@leomessi t amo eres mi ídolo ❤️🥺 dale me gusta a mi comentario porfa @leomessi 🥺🥺
11
sina.tmfootball
Please follow me
pic_count: 7
#####################################################################################################
https://instagram.fixc2-1.fna.fbcdn.net/v/t51.2885-15/e35/p1080x1080/84449291_172719960817410_1737062879184193005_n.jpg?_nc_ht=instagram.fixc2-1.fna.fbcdn.net&_nc_cat=106&_nc_ohc=ehviJwUZl7oAX_XIASW&oh=66b54551455da21ecb35aa43552556cd&oe=5F11B0EB
Photo by Leo Messi in Barcelona, Spain with @laureussport. Image may contain: 1 person, drink and outdoor
leomessi
Fue un verdadero honor recibir 

leomessi
HOY se estrena oficialmente la película original de Rakuten, #MessiCirqueTheMovie, el único documental que les permitirá conocer los secretos que esconde el espectáculo Messi10 by Cirque du Soleil (@messicirque). Espero que puedan disfrutarlo tanto como lo hice yo. Estará en exclusiva y de forma gratuita en @RakutenTV_ES

TODAY is the launch of #MessiCirqueTheMovie, Rakuten’s original documentary revealing the secrets behind Messi10 by Cirque du Soleil (@messicirque). I hope you enjoy watching it as much as I enjoyed making it! Exclusively available for free on @RakutenTV
count_comments  12
post_likes  1,420,027
1
sreerag_km__
😍😘😘
2
sahinali.kaya
Gt
3
sahinali.kaya
@sahinali.Kaya
4
arman.__.arya
جووووون چ پسری
5
ala_range_rover
👑👑👑👑👑👑
6
lizzyyourfantasy
❤️❤️
7
abdelkerimbaharmahamat
Bon photo man ami
8
tabiatbahmaie
👏👏👏
9
markariananri
🔥
10
nozanin_0070
❤️
11
nyrbol_2007
хорошо
pic_count: 15
######################################################################################

In [ ]:
/html[1]/body[1]/div[4]/div[2]/div[1]/article[1]/div[1]/div[1]/div[1]/div[1]/div[1]/img[1]
/html[1]/body[1]/div[4]/div[2]/div[1]/article[1]/div[1]/div[1]/div[1]/div[1]/div[1]/img[1]
/html[1]/body[1]/div[1]/section[1]/main[1]/article[1]/div[2]/div[1]/div[1]/div[1]

In [78]:
Insta_df_url

,Image Source,Image Alternate,posted by,Caption,No. of (Comments+replies),No. of likes,Users who commented,Comments on post,post hashtags,Date
0,https://instagram.fixc2-1.fna.fbcdn.net/v/t51....,"Photo shared by Leo Messi on June 13, 2020 tag...",leomessi,✈️📍Mallorca - @laliga,24,"6,387,075","[carles5puyol, themessistore, laliga, thetrill...","[💪💪💪💪, 🐐🐐🐐, 🐐😷🔥⚽️🔜, Best Soccer Player ⚽️❤️, G...",None,"June 13, 2020 tagging @3gerardpique, @antogrie..."
1,https://instagram.fixc2-1.fna.fbcdn.net/v/t51....,"Photo shared by Leo Messi on May 26, 2020 tagg...",leomessi,Listo para ⚽️ /// Ready for ⚽\n#Nemeziz,20,"3,623,216","[carles5puyol, themessistore, laliga, thetrill...","[💪💪💪💪, 🐐🐐🐐, 🐐😷🔥⚽️🔜, Best Soccer Player ⚽️❤️, G...",#Nemeziz,"May 26, 2020 tagging @adidasfootball"
2,https://instagram.fixc2-1.fna.fbcdn.net/v/t51....,Photo by Leo Messi in Ciutat Esportiva Joan Ga...,leomessi,User did not put any caption,20,"4,342,466","[carles5puyol, themessistore, laliga, thetrill...","[💪💪💪💪, 🐐🐐🐐, 🐐😷🔥⚽️🔜, Best Soccer Player ⚽️❤️, G...",None,
3,https://instagram.fixc2-1.fna.fbcdn.net/v/t51....,"Photo shared by Leo Messi on April 12, 2020 ta...",leomessi,Ayer terminaba la Semana Mundial de los trabaj...,12,"2,619,923","[carles5puyol, themessistore, laliga, thetrill...","[💪💪💪💪, 🐐🐐🐐, 🐐😷🔥⚽️🔜, Best Soccer Player ⚽️❤️, G...",##COVID19.#YoMeQuedoEnCasa#StayAtHome#Unicef,"April 12, 2020 tagging @unicef"
4,https://instagram.fixc2-1.fna.fbcdn.net/v/t51....,"Photo shared by Leo Messi on March 25, 2020 ta...",leomessi,#QuedateEnCasa 🏠\n#StayAtHome,12,"5,043,888","[carles5puyol, themessistore, laliga, thetrill...","[💪💪💪💪, 🐐🐐🐐, 🐐😷🔥⚽️🔜, Best Soccer Player ⚽️❤️, G...",#QuedateEnCasa#StayAtHome,"March 25, 2020 tagging @antonelaroccuzzo"
5,https://instagram.fixc2-1.fna.fbcdn.net/v/t51....,"Photo shared by Leo Messi on March 14, 2020 ta...",leomessi,Son días complicados para todo el mundo. Vivim...,13,"7,704,833","[carles5puyol, themessistore, laliga, thetrill...","[💪💪💪💪, 🐐🐐🐐, 🐐😷🔥⚽️🔜, Best Soccer Player ⚽️❤️, G...",#QuedateEnCasa#StayAtHome,"March 14, 2020 tagging @antonelaroccuzzo"
6,https://instagram.fixc2-1.fna.fbcdn.net/v/t51....,Photo by Leo Messi in Stadio San Paolo with @c...,leomessi,@championsleague ⚽️🏟,12,"4,193,344","[carles5puyol, themessistore, laliga, thetrill...","[💪💪💪💪, 🐐🐐🐐, 🐐😷🔥⚽️🔜, Best Soccer Player ⚽️❤️, G...",None,"league, and @miguelruizfcb"
7,https://instagram.fixc2-1.fna.fbcdn.net/v/t51....,"Photo by Leo Messi in Barcelona, Spain with @l...",leomessi,Fue un verdadero honor recibir el premio Laure...,12,"5,679,237","[carles5puyol, themessistore, laliga, thetrill...","[💪💪💪💪, 🐐🐐🐐, 🐐😷🔥⚽️🔜, Best Soccer Player ⚽️❤️, G...",#Laureus20#SportUnitesUs,", Spain with @laureussport"
8,https://instagram.fixc2-1.fna.fbcdn.net/v/t51....,"Photo shared by Leo Messi on January 26, 2020 ...",leomessi,No tengo palabras... Todo mi cariño para la fa...,12,"8,837,904","[carles5puyol, themessistore, laliga, thetrill...","[💪💪💪💪, 🐐🐐🐐, 🐐😷🔥⚽️🔜, Best Soccer Player ⚽️❤️, G...",None,"January 26, 2020 tagging @kobebryant"
9,https://instagram.fixc2-1.fna.fbcdn.net/v/t51....,"Photo by Leo Messi in Barcelona, Spain with @a...",leomessi,Gracias a todos por acompañarme esta tarde y a...,12,"3,366,421","[carles5puyol, themessistore, laliga, thetrill...","[💪💪💪💪, 🐐🐐🐐, 🐐😷🔥⚽️🔜, Best Soccer Player ⚽️❤️, G...",None,", Spain with @adidas_es"


In [80]:
Insta_df_url['Image Alternate'].iloc[10]

'Photo by Leo Messi in Paris, France with @fcbarcelona, @antonelaroccuzzo, @francefootball, and @afaseleccion. Image may contain: one or more people and people standing'

In [33]:
def postingInsta(username,pwd):
        ###################################### LOGIN #####################################    
    driver = webdriver.Chrome('chromedriver')
    driver.maximize_window()
    driver.get('https://www.instagram.com/')
    time.sleep(3)

    driver.find_element_by_xpath("//div[@class='EPjEi']/form/div[2]/div").click()

    keyboard.write(username)
    keyboard.press_and_release('tab') # go to pwd
    time.sleep(2)
    keyboard.write(pwd)
    keyboard.press_and_release('tab')# go to show button(not necessary)
    keyboard.press_and_release('tab')#go to login button
    keyboard.press_and_release('enter')#press enter to login
    time.sleep(5)
    #########################################
    for i in range(2):
        keyboard.press_and_release('tab')# to cancel a pop up of to turn on notifications
        keyboard.press_and_release('tab')
        keyboard.press_and_release('enter')
        time.sleep(2)
    time.sleep(1)
    keyboard.press_and_release('F12') #open dev tools
    time.sleep(1)
    keyboard.press_and_release('ctrl+shift+m')# toggle to mobile
    time.sleep(1)
    keyboard.press_and_release('F5')#refresh
    time.sleep(4)
    keyboard.press_and_release('F12')#close dev tools
    time.sleep(7)
    keyboard.press_and_release('tab')# to cancel a pop up of to turn on notifications
    keyboard.press_and_release('tab')
    keyboard.press_and_release('enter')
    driver.find_elements_by_xpath("//body//nav//div[3]//*[local-name()='svg']")[0].click()
    
    
    

In [34]:
postingInsta(username,pwd)

In [35]:
#downloading and saving files
    img_url='https://instagram.fixc2-1.fna.fbcdn.net/v/t51.2885-15/e35/104068437_119234286213828_6552583961155536986_n.jpg?_nc_ht=instagram.fixc2-1.fna.fbcdn.net&_nc_cat=100&_nc_ohc=cdTkheSySL0AX-AjDaf&oh=ab73dd3f482a397905f90577b2e7eb8c&oe=5F1481C4'
    r = requests.get(img_url)
    with open("C://Users//ILLUMINATI//Downloads//pic.jpg",'wb') as f:
        f.write(r.content)



In [58]:
def Follow_people(username,pwd):
    ###################################### LOGIN #####################################    
    driver = webdriver.Chrome('chromedriver')
    driver.maximize_window()
    driver.get('https://www.instagram.com/')
    time.sleep(3)

    driver.find_element_by_xpath("//div[@class='EPjEi']/form/div[2]/div").click()

    keyboard.write(username)
    keyboard.press_and_release('tab') # go to pwd
    time.sleep(2)
    keyboard.write(pwd)
    keyboard.press_and_release('tab')# go to show button(not necessary)
    keyboard.press_and_release('tab')#go to login button
    keyboard.press_and_release('enter')#press enter to login
    time.sleep(5)    
    ################################## Put in Username to follow##########################
    username_list=['vermakanchan760','amandeepgill52','amit_madaan8296']
    i=0
    while i!=username_list:
        try:
#             print(1)
            print(i)
            driver.find_element_by_xpath("//div[@class='LWmhU _0aCwM']/div").click()
            keyboard.write(username_list[i])
            time.sleep(3)
            keyboard.press_and_release('tab')
            keyboard.press_and_release('enter')
            time.sleep(3)
    #         driver.find_element_by_xpath("//div[@class='nZSzR']").click()
    #         for i in range(2):
            keyboard.press_and_release('tab')
            time.sleep(3)
            keyboard.press_and_release('tab')
            time.sleep(3)
#             keyboard.press_and_release('tab')
#             time.sleep(3)
            keyboard.press_and_release('enter')    
            time.sleep(3)
#             print(2)
            i+=1
        except:
#             print(3)
            if i>=len(username_list):
                return 0
            keyboard.press_and_release('tab')
            keyboard.press_and_release('tab')
            keyboard.press_and_release('enter')
            keyboard.press_and_release('esc')
            driver.get('https://www.instagram.com/')
            
            
    

In [59]:
#to follow people from DB on insta
Follow_people(username,pwd)

0
1
2
2
2
3


0